In [67]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

# Packages

In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import datetime
from datetime import datetime, timedelta
#import plotly.graph_objs as go
import matplotlib.pyplot as plt
import csv
import seaborn as sns
%matplotlib inline
sns.set()
from PIL import Image

Let $j$ denoting a combination of age and gender categories (strata). If $j$ has $O_{ij}$ observed events out of a population $Pop_{ij}$, then expected number of cases $E_i$ for an area, $ i$, represents the prior spatial distribution of the disease. This distribution is established by the population structure, which varies according to the combination of age and gender categories of the whole population. In such a way, the expected number of events in this strata is given by:
\begin{equation}\label{eq1}
E_{ij} = Pop_{ij}\frac{\sum_{i=1}^{n}O_{ij}}{\sum_{i=1}^{n}Pop_{ij}},
\end{equation}
and the total number of expected events in area $i$	 is just:
\begin{equation}\label{eq2}
E_{i} = \sum_{j=1}^{J}E_{ij},
\end{equation}
where $n$ is the number of aggregated count and $J$ the number of categories \cite{Leyland2005}. 

The relative risk within an area is given as the ratio between the observed and expected number of cases of the disease. This ratio is the so called standard incidence ratio and is given by:
\begin{equation}\label{eq3}
\theta_i = \frac{O_{i}}{E_{i}}.
\end{equation}
If the observed number of cases equals the expected number, the $\theta$ is 1. If more cases are observed than expected, then $\theta$ is greater than 1. If fewer cases are observed than expected, the $\theta$ is less than 1. 



# Population data to calculate the risk

In [3]:
pop = pd.read_csv('/Users/julianeoliveira/Desktop/Modelling TaskForce Arboviruses/Data/pop.csv')

In [4]:
pop = pop.astype(int)

In [5]:
pop.head()

,Unnamed: 0,MUNIC_RES,ANO,SEXO,SITUACAO,FXETARIA,POPULACAO
0,0,110001,2012,1,3,4,187
1,1,110001,2012,1,3,4,186
2,2,110001,2012,1,3,4,187
3,3,110001,2012,1,3,4,190
4,4,110001,2012,1,3,4,193


# Confirmed dengue dataset

In [6]:
df = pd.read_csv('/Users/julianeoliveira/Desktop/Modelling TaskForce Arboviruses/Data/dengue_conf_clean.csv',low_memory = False)

In [7]:
df['dt_sin_pri'] = pd.to_datetime(df['dt_sin_pri'], errors = 'coerce')
df['dt_notific'] = pd.to_datetime(df['dt_notific'])
df['dt_nasc'] = pd.to_datetime(df['dt_nasc'])

In [8]:
df['sem_pri_correct'] = df['dt_sin_pri'].dt.strftime('%Y-%W')

In [9]:
df['month'] = df['dt_sin_pri'].dt.strftime('%Y-%m')

## Filter desired variables

In [10]:
data = df.filter(['codmunres','dt_sin_pri','year','month','sem_pri','sem_pri_correct','cs_sexo','faixa','count'])

In [11]:
dados = data.groupby(['codmunres','year','month','sem_pri_correct','cs_sexo','faixa'])['count'].sum().reset_index()

In [12]:
dados.head()

,codmunres,year,month,sem_pri_correct,cs_sexo,faixa,count
0,110000.0,2015,2015-12,2015-52,1,4,1
1,110001.0,2010,2010-01,2010-02,2,7074,1
2,110001.0,2010,2010-01,2010-03,2,5559,1
3,110001.0,2010,2010-02,2010-06,1,1014,2
4,110001.0,2010,2010-02,2010-06,1,3539,1


In [13]:
dados.groupby(['year'])['count'].sum()

year
2010     869202
2011     600937
2012     401708
2013    1188781
2014     475207
2015    1386871
2016     479362
2017     167600
2018     146490
2019    1280783
Name: count, dtype: int64

In [14]:
df.year.value_counts()

2015    1387045
2019    1280791
2013    1188923
2010     869246
2011     601017
2016     479382
2014     475282
2012     401768
2017     167604
2018     146494
Name: year, dtype: int64

## Standard incidence ratio of confirmed dengue

In [15]:
teste_pop = pop.groupby(['MUNIC_RES','ANO','SEXO','FXETARIA'])['POPULACAO'].sum().reset_index()
teste_pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2012,1,4,943
1,110001,2012,1,509,1058
2,110001,2012,1,1014,1239
3,110001,2012,1,1519,1343
4,110001,2012,1,2024,1090


In [16]:
table_pop = teste_pop.groupby(['SEXO','FXETARIA'])['POPULACAO'].sum().reset_index()
table_pop.rename(columns = {"SEXO": "cs_sexo", "FXETARIA": "faixa"}, inplace = True)
table_pop.head()

,cs_sexo,faixa,POPULACAO
0,1,4,7143389
1,1,509,7758474
2,1,1014,8875967
3,1,1519,8704999
4,1,2024,8782606


In [17]:
teste = dados

In [18]:
#remove not determinated from sex
teste = teste[teste.cs_sexo != 'I']
teste["cs_sexo"] = teste["cs_sexo"].astype(int)

<ipython-input-18-6795e7026f25>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teste["cs_sexo"] = teste["cs_sexo"].astype(int)


In [19]:
teste.codmunres = teste['codmunres'].astype(int)
teste.head()

/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,codmunres,year,month,sem_pri_correct,cs_sexo,faixa,count
0,110000,2015,2015-12,2015-52,1,4,1
1,110001,2010,2010-01,2010-02,2,7074,1
2,110001,2010,2010-01,2010-03,2,5559,1
3,110001,2010,2010-02,2010-06,1,1014,2
4,110001,2010,2010-02,2010-06,1,3539,1


In [20]:
#create data set one for week and another for month
monthly_data = teste.groupby(["month", "cs_sexo", "faixa"])["count"].sum().reset_index(name = "cases")
weekly_data = teste.groupby(["sem_pri_correct", "cs_sexo", "faixa"])["count"].sum().reset_index(name = "cases")

In [21]:
#merge pop to booth data
monthly_data = monthly_data.merge(right = table_pop, on = ["cs_sexo", "faixa"])
weekly_data = weekly_data.merge(right = table_pop, on = ["cs_sexo", "faixa"])

In [22]:
monthly_data.head()

,month,cs_sexo,faixa,cases,POPULACAO
0,2009-12,1,4,29,7143389
1,2010-01,1,4,1897,7143389
2,2010-02,1,4,2445,7143389
3,2010-03,1,4,3854,7143389
4,2010-04,1,4,3133,7143389


In [23]:
weekly_data.head()

,sem_pri_correct,cs_sexo,faixa,cases,POPULACAO
0,2009-52,1,4,29,7143389
1,2010-00,1,4,155,7143389
2,2010-01,1,4,422,7143389
3,2010-02,1,4,359,7143389
4,2010-03,1,4,443,7143389


####  Compute $\frac{\sum_{i=1}^{n}O_{ij}}{\sum_{i=1}^{n}Pop_{ij}}$

In [24]:
monthly_data["SOij_Spij"] = monthly_data["cases"]/monthly_data["POPULACAO"]

weekly_data["SOij_Spij"] = weekly_data["cases"]/weekly_data["POPULACAO"]

In [25]:
#check if the SOij_Spij was computed correclty
monthly_data[monthly_data["month"] == "2009-12"].head()

,month,cs_sexo,faixa,cases,POPULACAO,SOij_Spij
0,2009-12,1,4,29,7143389,0.000004
121,2009-12,1,509,33,7758474,0.000004
242,2009-12,1,1014,40,8875967,0.000005
362,2009-12,1,1519,62,8704999,0.000007
483,2009-12,1,2024,96,8782606,0.000011


In [26]:
#run for a specific month just for confirm values
SOij_Spij_month = []
for month in ["2009-12"]:
    print(month)
    
    
    #df_month = teste[teste.month == month]
    #table_month = pd.pivot_table(df_month, values='count', index=['codmunres'], columns=['cs_sexo','faixa']).fillna(0)
    #resulted_dataframe = table_year.sum().reset_index()
    
    datamonth = teste[teste.month == month].groupby(['cs_sexo','faixa'])['count'].sum().reset_index()
    
    
    datamonth['SOij_Spij'] = datamonth['count']/table_pop.POPULACAO
   
    
    SOij_Spij_month.append(datamonth)
    
    

2009-12


In [27]:
datamonth.head()

,cs_sexo,faixa,count,SOij_Spij
0,1,4,29,0.000004
1,1,509,33,0.000004
2,1,1014,40,0.000005
3,1,1519,62,0.000007
4,1,2024,96,0.000011


#### Compute  $E_{ij} = Pop_{ij}\frac{\sum_{i=1}^{n}O_{ij}}{\sum_{i=1}^{n}Pop_{ij}}$

In [28]:
teste_pop.rename(columns = {"MUNIC_RES": "codmunres", "SEXO": "cs_sexo", "FXETARIA": "faixa"}, inplace = True)
teste_pop.head()

,codmunres,ANO,cs_sexo,faixa,POPULACAO
0,110001,2012,1,4,943
1,110001,2012,1,509,1058
2,110001,2012,1,1014,1239
3,110001,2012,1,1519,1343
4,110001,2012,1,2024,1090


In [29]:
#create a monthly series for each city
cities_monthly_series = teste.groupby(["codmunres", "month", "cs_sexo", "faixa"])["count"].sum().reset_index(name = "cases")

#create a weekly series for each city
cities_weekly_series = teste.groupby(["codmunres", "sem_pri_correct", "cs_sexo", "faixa"])["count"].sum().reset_index(name = "cases")

In [30]:
cities_monthly_series.head()

,codmunres,month,cs_sexo,faixa,cases
0,110000,2015-12,1,4,1
1,110001,2010-01,2,5559,1
2,110001,2010-01,2,7074,1
3,110001,2010-02,1,509,1
4,110001,2010-02,1,1014,2


In [31]:
cities_weekly_series.head()

,codmunres,sem_pri_correct,cs_sexo,faixa,cases
0,110000,2015-52,1,4,1
1,110001,2010-02,2,7074,1
2,110001,2010-03,2,5559,1
3,110001,2010-06,1,1014,2
4,110001,2010-06,1,3539,1


In [32]:
monthly_data.head()

,month,cs_sexo,faixa,cases,POPULACAO,SOij_Spij
0,2009-12,1,4,29,7143389,0.000004
1,2010-01,1,4,1897,7143389,0.000266
2,2010-02,1,4,2445,7143389,0.000342
3,2010-03,1,4,3854,7143389,0.000540
4,2010-04,1,4,3133,7143389,0.000439


In [33]:
#merge population data
cities_monthly_series = cities_monthly_series.merge(right = teste_pop, on = ["codmunres", "cs_sexo", "faixa"])
cities_weekly_series = cities_weekly_series.merge(right = teste_pop, on = ["codmunres", "cs_sexo", "faixa"])

In [34]:
# merge the city series with the data with SOij_Spij calculated
cities_monthly_series = cities_monthly_series.merge(right = monthly_data, on = ["month", "cs_sexo", "faixa"])
cities_weekly_series = cities_weekly_series.merge(right = weekly_data, on = ["sem_pri_correct", "cs_sexo", "faixa"])

In [35]:
cities_monthly_series['Eij'] = cities_monthly_series['POPULACAO_x']*cities_monthly_series['SOij_Spij']

cities_weekly_series['Eij'] = cities_weekly_series['POPULACAO_x']*cities_weekly_series['SOij_Spij']

In [36]:
cities_monthly_series.head()

,codmunres,month,cs_sexo,faixa,cases_x,ANO,POPULACAO_x,cases_y,POPULACAO_y,SOij_Spij,Eij
0,110001,2010-01,2,5559,1,2012,425,2013,4441120,0.000453,0.192637
1,110002,2010-01,2,5559,1,2012,1523,2013,4441120,0.000453,0.690321
2,110004,2010-01,2,5559,13,2012,1441,2013,4441120,0.000453,0.653153
3,110008,2010-01,2,5559,5,2012,187,2013,4441120,0.000453,0.084760
4,110009,2010-01,2,5559,2,2012,506,2013,4441120,0.000453,0.229352


In [37]:
cities_weekly_series.head()

,codmunres,sem_pri_correct,cs_sexo,faixa,cases_x,ANO,POPULACAO_x,cases_y,POPULACAO_y,SOij_Spij,Eij
0,110001,2010-02,2,7074,1,2012,175,175,2103802,0.000083,0.014557
1,110004,2010-02,2,7074,5,2012,613,175,2103802,0.000083,0.050991
2,110009,2010-02,2,7074,1,2012,198,175,2103802,0.000083,0.016470
3,110010,2010-02,2,7074,1,2012,264,175,2103802,0.000083,0.021960
4,110011,2010-02,2,7074,1,2012,387,175,2103802,0.000083,0.032192


#### Compute $E_{i} = \sum_{j=1}^{J}E_{ij}$

In [38]:
cities_monthly_series= cities_monthly_series.groupby(['codmunres','month'])['Eij'].sum().reset_index(name = "Ei")

cities_weekly_series= cities_weekly_series.groupby(['codmunres','sem_pri_correct'])['Eij'].sum().reset_index(name = "Ei")

In [41]:
cities_weekly_series

,codmunres,sem_pri_correct,Ei
0,110001,2010-02,0.014557
1,110001,2010-03,0.040288
2,110001,2010-06,0.389612
3,110001,2010-07,0.513993
4,110001,2010-08,0.263554
...,...,...,...
403270,530010,2019-47,67.777408
403271,530010,2019-48,76.450353
403272,530010,2019-49,87.977635
403273,530010,2019-50,73.891245


In [39]:
#create a monthly series for each city
cities_monthly_series2 = teste.groupby(["codmunres", "month"])["count"].sum().reset_index(name = "cases")

#create a weekly series for each city
cities_weekly_series2 = teste.groupby(["codmunres", "sem_pri_correct"])["count"].sum().reset_index(name = "cases")

In [40]:
cities_monthly_series2.head()

,codmunres,month,cases
0,110000,2015-12,1
1,110001,2010-01,2
2,110001,2010-02,9
3,110001,2010-05,1
4,110001,2010-11,1


In [42]:
cities_monthly_series = cities_monthly_series.merge(right = cities_monthly_series2, on = ['codmunres',"month"])
cities_weekly_series = cities_weekly_series.merge(right = cities_weekly_series2, on = ['codmunres',"sem_pri_correct"])

#### Compute the $\theta_i = \frac{O_{i}}{E_{i}}$

In [45]:
cities_monthly_series['SIR'] = cities_monthly_series['cases']/cities_monthly_series['Ei']
cities_weekly_series['SIR'] = cities_weekly_series['cases']/cities_weekly_series['Ei']

In [46]:
cities_monthly_series.head()

,codmunres,month,Ei,cases,SIR
0,110001,2010-01,0.257187,2,7.776442
1,110001,2010-02,4.517608,9,1.992205
2,110001,2010-05,0.843320,1,1.185789
3,110001,2010-11,0.011109,1,90.016818
4,110001,2011-03,0.427670,1,2.338249


In [47]:
cities_weekly_series.head()

,codmunres,sem_pri_correct,Ei,cases,SIR
0,110001,2010-02,0.014557,1,68.695576
1,110001,2010-03,0.040288,1,24.821126
2,110001,2010-06,0.389612,4,10.266615
3,110001,2010-07,0.513993,3,5.836655
4,110001,2010-08,0.263554,2,7.588585


In [48]:
codigo_muni = pd.read_excel('/Users/julianeoliveira/Desktop/hard_disc/Datalake/POPULACAO/População/POP_python/DTB_BRASIL_MUNICIPIO.xls')

In [49]:
codigo_muni

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município
0,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso
3,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste
4,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes
...,...,...,...,...,...,...,...,...,...
5565,52,Goiás,5,Sul Goiano,16,Pires do Rio,22005,5222005,Vianópolis
5566,52,Goiás,5,Sul Goiano,15,Meia Ponte,22054,5222054,Vicentinópolis
5567,52,Goiás,4,Leste Goiano,12,Entorno de Brasília,22203,5222203,Vila Boa
5568,52,Goiás,4,Leste Goiano,12,Entorno de Brasília,22302,5222302,Vila Propício


In [50]:
codigo_muni.rename(columns={'Código Município Completo':'codmuni_full'}, 
                 inplace=True);

In [53]:
codigo_muni['codmunres'] = codigo_muni.codmuni_full.floordiv(10)

In [61]:
codigo_muni =codigo_muni.drop(columns=['codmuni'])

In [62]:
cities_monthly_series=cities_monthly_series.merge(right = codigo_muni, on = ["codmunres"])

cities_weekly_series=cities_weekly_series.merge(right = codigo_muni, on = ["codmunres"])

In [70]:
cities_weekly_series.head()

,codmunres,sem_pri_correct,Ei,cases,SIR,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,codmuni_full,Nome_Município
0,110001,2010-02,0.014557,1,68.695576,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,110001,2010-03,0.040288,1,24.821126,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
2,110001,2010-06,0.389612,4,10.266615,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
3,110001,2010-07,0.513993,3,5.836655,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
4,110001,2010-08,0.263554,2,7.588585,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste


#### Save final datasets

In [68]:
cities_monthly_series.to_csv('/Users/julianeoliveira/Desktop/github/Datasets from the gitcomputations/den_conf_cities_monthly_series.csv')

In [69]:
cities_weekly_series.to_csv('/Users/julianeoliveira/Desktop/github/Datasets from the gitcomputations/den_conf_cities_weekly_series.csv')